## **1️⃣ Guardar y Cargar el Estado de un Agente**

Cada **agente** en AgentChat tiene un **estado** que incluye su historial de conversación. Podemos **guardar** y **cargar** este estado con los métodos `save_state()` y `load_state()`.

### **Ejemplo: Guardar y Restaurar el Estado de un Agente**

In [1]:
import json
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Crear un agente asistente
assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a helpful assistant",
    model_client=OpenAIChatCompletionClient(model="gpt-4o"),
)

# Enviar un mensaje inicial
response = await assistant_agent.on_messages(
    [TextMessage(content="Write a 3-line poem about a lake", source="user")],
    CancellationToken()
)
print("🔹 Respuesta del agente:", response.chat_message.content)

# Guardar el estado del agente
agent_state = await assistant_agent.save_state()
print("🔹 Estado guardado:", json.dumps(agent_state, indent=2))

# Crear un nuevo agente y restaurar su estado
new_assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a helpful assistant",
    model_client=OpenAIChatCompletionClient(model="gpt-4o"),
)
await new_assistant_agent.load_state(agent_state)

# Preguntar algo sobre la conversación anterior
response = await new_assistant_agent.on_messages(
    [TextMessage(content="What was the last line of the previous poem you wrote?", source="user")],
    CancellationToken()
)
print("🔹 Respuesta del agente restaurado:", response.chat_message.content)


🔹 Respuesta del agente: Silent ripples dance,  
Moonlight kisses the stillness,  
Secrets held below.
🔹 Estado guardado: {
  "type": "AssistantAgentState",
  "version": "1.0.0",
  "llm_context": {
    "messages": [
      {
        "content": "Write a 3-line poem about a lake",
        "source": "user",
        "type": "UserMessage"
      },
      {
        "content": "Silent ripples dance,  \nMoonlight kisses the stillness,  \nSecrets held below.",
        "source": "assistant_agent",
        "type": "AssistantMessage"
      }
    ]
  }
}
🔹 Respuesta del agente restaurado: The last line of the poem was "Secrets held below."


## **2️⃣ Guardar y Restaurar el Estado de un Equipo**

Cuando trabajamos con **equipos de agentes**, podemos **guardar y cargar el estado del equipo completo**, incluyendo todos los agentes dentro de él.

### **Ejemplo: Guardar y Restaurar un Equipo de Agentes**

In [2]:
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

# Crear un equipo de agentes
agent_team = RoundRobinGroupChat(
    [assistant_agent], termination_condition=MaxMessageTermination(max_messages=2)
)

# Ejecutar el equipo con una tarea
await Console(agent_team.run_stream(task="Write a 3-line poem about the sea"))

# Guardar el estado del equipo
team_state = await agent_team.save_state()
print("🔹 Estado del equipo guardado:", json.dumps(team_state, indent=2))

# Resetear el equipo (pierde la memoria)
await agent_team.reset()

# Preguntar sobre el poema sin restaurar el estado
await Console(agent_team.run_stream(task="What was the last line of the poem you wrote?"))

# Restaurar el estado y volver a preguntar
await agent_team.load_state(team_state)
await Console(agent_team.run_stream(task="What was the last line of the poem you wrote?"))


---------- user ----------
Write a 3-line poem about the sea
---------- assistant_agent ----------
Waves crash with a roar,  
Endless blue stretches beyond,  
Whispers of the deep.
🔹 Estado del equipo guardado: {
  "type": "TeamState",
  "version": "1.0.0",
  "agent_states": {
    "group_chat_manager/e4ec37bf-8ac7-4196-b82e-3cf85dc6ed80": {
      "type": "RoundRobinManagerState",
      "version": "1.0.0",
      "message_thread": [
        {
          "source": "user",
          "models_usage": null,
          "content": "Write a 3-line poem about the sea",
          "type": "TextMessage"
        },
        {
          "source": "assistant_agent",
          "models_usage": {
            "prompt_tokens": 65,
            "completion_tokens": 22
          },
          "content": "Waves crash with a roar,  \nEndless blue stretches beyond,  \nWhispers of the deep.",
          "type": "TextMessage"
        }
      ],
      "current_turn": 0,
      "next_speaker_index": 0
    },
    "collect_o

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What was the last line of the poem you wrote?', type='TextMessage'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=109, completion_tokens=17), content='The last line of the poem I wrote is "Whispers of the deep."', type='TextMessage')], stop_reason='Maximum number of messages 2 reached, current message count: 2')

## **3️⃣ Guardar y Cargar Estados desde Archivos JSON**

Podemos **guardar el estado en un archivo JSON** para recuperarlo más tarde.

### **Ejemplo: Guardar Estado en Archivo JSON**

In [ ]:
# Guardar estado en un archivo
with open("team_state.json", "w") as f:
    json.dump(team_state, f)

# Cargar estado desde un archivo
with open("team_state.json", "r") as f:
    loaded_team_state = json.load(f)

# Restaurar el equipo desde el archivo
await agent_team.load_state(loaded_team_state)
await Console(agent_team.run_stream(task="What was the last line of the poem you wrote?"))
